# Settings

In [ ]:
seed = 8018
drop_duplicates_for_projection = True

# Prep Model

In [ ]:
# define the network
import tensorflow as tf
dims = (8, 8, 13)
n_components = 2
encoder = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=dims),
    tf.keras.layers.Conv2D(
        filters=16, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Conv2D(
        filters=32, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Conv2D(
        filters=128, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Conv2D(
        filters=256, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=100),
    tf.keras.layers.Dense(units=100),
    tf.keras.layers.Dense(units=100),
    tf.keras.layers.Dense(units=n_components),
])
encoder.summary()

# Training Parameters

In [ ]:
keras_fit_kwargs = {"callbacks": [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=10**-2,
        patience=10,
        verbose=1,
    )
]}

# Prep Data

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('all_openings.csv')

# One-hot encode the chessboard columns
expected_categories = ['wp', 'wr', 'wn', 'wb', 'wq', 'wk', 'bp', 'br', 'bn', 'bb', 'bq', 'bk', '']

# Define chessboard columns
chessboard_columns = [f"{col}{row}" for row in range(1, 9) for col in "abcdefgh"]

for column in chessboard_columns:
    df[column] = pd.Categorical(df[column], categories=expected_categories)


training_data_df = df[chessboard_columns]

training_data_df = pd.get_dummies(training_data_df, columns=chessboard_columns)

# Combine the one-hot encoded chessboard DataFrame with the rest of the metadata
# First, drop the original chessboard columns from the main DataFrame to avoid duplicates
metadata_df = df.drop(columns=chessboard_columns)
# Then, concatenate the encoded chessboard DataFrame with the metadata DataFrame
combined_df = pd.concat([metadata_df, training_data_df], axis=1)

In [ ]:
if drop_duplicates_for_projection:
    # drop duplicates for projection, use the column names from chessboard_dummies_df for finding duplicates
    print("Length before dropping duplicates:", len(training_data_df))
    training_data_df.drop_duplicates(subset=training_data_df.columns, inplace=True)
    print("Length after dropping duplicates:", len(training_data_df))

In [ ]:
import numpy as np

# Load and preprocess the data correctly for the encoder
# get numpy data out of training_data_df
train_data = training_data_df.to_numpy()

# Verify the shape
print("Train shape:", train_data.shape)

# Generate embeddings using the encoder just to verify the shape
train_embeddings = encoder.predict(train_data.reshape(-1, 8, 8, 13))

# Reducer

In [ ]:
from umap.parametric_umap import ParametricUMAP

reducer = ParametricUMAP(
    verbose=True,
    keras_fit_kwargs = keras_fit_kwargs,
    encoder=encoder,
    dims=dims,
    random_state=seed,
    n_training_epochs=20)

# Fit

In [ ]:
import colorama
reducer = ParametricUMAP(encoder=encoder, dims=dims)
print("Before fitting, check dims:", dims)
print("Reducer expected input shape:", reducer.dims)
# Attempt to fit and transform
try:
    embedding = reducer.fit_transform(train_data.reshape((train_data.shape[0], -1)))
    print(f"Embedding shape after fit_transform: {embedding.shape}")
except Exception as e:
    print(f"{colorama.Fore.RED}Error during fit_transform: {e}{colorama.Style.RESET_ALL}")

In [ ]:
# from umap.parametric_umap import load_ParametricUMAP
# embedder = load_ParametricUMAP('/your/path/here')

In [ ]:
# create unique filename based on date and time
from datetime import datetime
now = datetime.now()
reducer.save('./embeddings/parametric_umap_embeddings_' + now.strftime("%Y-%m-%d_%H-%M-%S"))

In [ ]:

# list of chess board dummy columns (to be used for hashing) = training_data_df columns
chessboard_dummy_columns = training_data_df.columns

# Add Projection Coordinates to Training Data DataFrame
training_data_df["x"] = embedding[:, 0]
training_data_df["y"] = embedding[:, 1]

# Generate a unique hash for each row's configuration in both DataFrames
def generate_row_hash(df, prefix_columns):
    return df[prefix_columns].apply(lambda x: hash(tuple(x)), axis=1)

# Generate hashes for both DataFrames
training_data_hash = generate_row_hash(training_data_df, chessboard_dummy_columns)
combined_data_hash = generate_row_hash(combined_df, chessboard_dummy_columns)  # Only the one-hot encoded columns

# Add these hashes as a column to both DataFrames
training_data_df['config_hash'] = training_data_hash
combined_df['config_hash'] = combined_data_hash

# Merge the x, y coordinates from training_data_df to combined_df based on the hash
combined_df = combined_df.merge(training_data_df[['config_hash', 'x', 'y']], on='config_hash', how='left')

# Cleanup if necessary (drop the hash column if no longer needed)
combined_df.drop(columns=['config_hash'], inplace=True)

In [ ]:
# drop x_x and y_x
combined_df.drop(columns=['x_x', 'y_x'], inplace=True)
# rename x_y to x, and y_y to y
combined_df.rename(columns={'x_y': 'x', 'y_y': 'y'}, inplace=True)

In [ ]:
# replace the dummy columns with the original non-one-hot encoded columns
# First, drop the one-hot encoded columns
combined_df.drop(columns=chessboard_dummy_columns, inplace=True)

# Then, concatenate the original chessboard columns
combined_df[chessboard_columns] = df[chessboard_columns]


In [ ]:
# store csv as all_openings_projected_<timestamp>.csv
combined_df.to_csv('all_openings_projected_' + now.strftime("%Y-%m-%d_%H-%M-%S") + '.csv', index=False)

In [ ]:
# rename opening_type to algo, Result to cp, game_number to line
combined_df.rename(columns={'opening_type': 'algo', 'Result': 'cp', 'game_number': 'line'}, inplace=True)

In [ ]:
# store csv as all_openings_projected_PSE_format_<timestamp>.csv
combined_df.to_csv('all_openings_projected_PSE_format' + now.strftime("%Y-%m-%d_%H-%M-%S") + '.csv', index=False)
```